In [1]:
import numpy as np

class GuitarDegreeCalculator:
    def __init__(self, G1, G2, current_string_number, current_fret_number, current_degree, increase_degree):
        self.G1 = G1
        self.G2 = G2
        self.current_string_number = int(current_string_number)
        self.current_fret_number = int(current_fret_number)
        self.current_degree = int(current_degree)
        self.increase_degree = int(increase_degree)
        self.notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        self.standard_tuning = ['E', 'A', 'D', 'G', 'B', 'E']  # Standard tuning from 6th to 1st string

    def calculate_note_name(self, string_number, fret_number, degree_type):
        base_note = self.standard_tuning[6 - string_number]
        note_index = (self.notes.index(base_note) + fret_number) % len(self.notes)
        return self.notes[note_index]

    def determine_degree_type(self, after_degree):
        if self.increase_degree == 1:
            return "완전"
        elif self.increase_degree == 2:
            return "단" if after_degree in [4, 1] else "장"
        elif self.increase_degree == 3:
            return "장" if after_degree in [3, 6, 7] else "단"
        elif self.increase_degree == 4:
            return "증" if after_degree == 7 else "완전"
        elif self.increase_degree == 5:
            return "감" if after_degree == 4 else "완전"
        elif self.increase_degree == 6:
            return "장" if after_degree in [6, 7, 2, 3] else "단"
        elif self.increase_degree == 7:
            return "장" if after_degree in [7, 3] else "단"

    def calculate_fret_and_string(self, drop_string_number, degree_type):
        after_degree = (self.current_degree + (self.increase_degree - 1)) % 7
        after_degree = after_degree if after_degree != 0 else 7

        degree_index = after_degree - 1
        if drop_string_number >= 0 and drop_string_number <= 5:
            if self.current_string_number in [6, 5, 4, 3]:
                after_fret_number = self.calculate_fret_for_lower_strings(drop_string_number, degree_type, degree_index)
            elif self.current_string_number == 2 and drop_string_number <= 1:
                after_fret_number = self.calculate_fret_for_string_two(drop_string_number, degree_type)
            elif self.current_string_number == 1 and drop_string_number == 0:
                after_fret_number = self.calculate_fret_for_string_one(drop_string_number, degree_type)
            else:
                return None, None, None, None, None

            after_string_number = self.current_string_number - drop_string_number
            note_name = self.calculate_note_name(after_string_number, after_fret_number, degree_type)
            return after_fret_number, after_string_number, after_degree, note_name, degree_type

        return None, None, None, None, None

    def calculate_fret_for_lower_strings(self, drop_string_number, degree_type, degree_index):
        if drop_string_number in [4, 5]:
            if self.increase_degree == 1:
                return self.current_fret_number + self.G1[degree_index, drop_string_number] + self.G2[0, 0]
            else:
                return self.calculate_fret_based_on_degree_type(drop_string_number, degree_type)
        else:
            if self.increase_degree == 1:
                return self.current_fret_number + self.G1[degree_index, drop_string_number] + self.G2[0, 1]
            else:
                return self.calculate_fret_based_on_degree_type(drop_string_number, degree_type, second=True)

    def calculate_fret_based_on_degree_type(self, drop_string_number, degree_type, second=False):
        column_index = 1 if second else 0
        if degree_type == '장':
            return self.current_fret_number + self.G1[:, drop_string_number][1] + self.G2[0, column_index]
        elif degree_type == '단':
            return self.current_fret_number + self.G1[:, drop_string_number][2] + self.G2[1, column_index]
        elif degree_type == '완전':
            return self.current_fret_number + self.G1[:, drop_string_number][0] + self.G2[0, column_index]
        elif degree_type == '증':
            return self.current_fret_number + self.G1[:, drop_string_number][3] + self.G2[2, column_index]
        elif degree_type == '감':
            return self.current_fret_number + self.G1[:, drop_string_number][4] + self.G2[1, column_index]

    def calculate_fret_for_string_two(self, drop_string_number, degree_type):
        if degree_type == '장':
            return self.current_fret_number + self.G1[drop_string_number, 1] + self.G2[0, 1]
        elif degree_type == '단':
            return self.current_fret_number + self.G1[drop_string_number, 2] + self.G2[1, 1]
        elif degree_type == '완전':
            return self.current_fret_number + self.G1[drop_string_number, 0] + self.G2[0, 1]
        elif degree_type == '증':
            return self.current_fret_number + self.G1[drop_string_number, 3] + self.G2[2, 1]
        elif degree_type == '감':
            return self.current_fret_number + self.G1[drop_string_number, 4] + self.G2[1, 1]

    def calculate_fret_for_string_one(self, drop_string_number, degree_type):
        if degree_type == '장':
            return self.current_fret_number + self.G1[drop_string_number, 1] + self.G2[0, 0]
        elif degree_type == '단':
            return self.current_fret_number + self.G1[drop_string_number, 2] + self.G2[1, 0]
        elif degree_type == '완전':
            return self.current_fret_number + self.G1[drop_string_number, 0] + self.G2[0, 0]
        elif degree_type == '증':
            return self.current_fret_number + self.G1[drop_string_number, 3] + self.G2[2, 0]
        elif degree_type == '감':
            return self.current_fret_number + self.G1[drop_string_number, 4] + self.G2[1, 0]


def run_calculations(G1, G2, current_string_number, current_fret_number, current_degree, increase_degree):
    calculator = GuitarDegreeCalculator(G1, G2, current_string_number, current_fret_number, current_degree, increase_degree)

    for drop_string_number in range(6):
        degree_type = calculator.determine_degree_type((calculator.current_degree + (calculator.increase_degree - 1)) % 7)
        after_fret_number, after_string_number, after_degree, note_name, degree_type = calculator.calculate_fret_and_string(drop_string_number, degree_type)
        if after_fret_number is not None:
            print(f"drop_string_number: {drop_string_number}, 프렛 번호: {after_fret_number}, 줄 번호: {after_string_number}, 도수: {after_degree}, 음이름: {note_name}, 음정종류: {degree_type}") 
        else:
            print(f"drop_string_number: {drop_string_number}, 조건에 맞는 계산 결과가 없습니다.")

    print()
    print(f"현재 줄 번호: {current_string_number}")
    print(f"현재 프렛 번호: {current_fret_number}")
    print(f"현재 도수: {current_degree}")
    print(f"올릴 도수: {increase_degree}")

# Example usage (for testing purposes)
if __name__ == "__main__":
    G1 = np.array([[1, 3, 5, 6, 8, 10, 12],
                   [-4, -2, 0, 1, 3, 5, 7],
                   [-9, -7, -5, -4, -2, 0, 2],
                   [-14, -12, -10, -9, -7, -5, -3],
                   [-19, -17, -15, -14, -12, -10, -8],
                   [-24, -22, -20, -19, -17, -15, -13]])

    G2 = np.array([[0, -1],
                   [-1, -2],
                   [1, 0]])

    current_string_number = input('현재 줄 번호를 입력하세요: ')
    current_fret_number = input('현재 프렛 번호를 입력하세요: ')
    current_degree = input('현재 도수를 입력하세요: ')
    increase_degree = input('올릴 도수를 입력하세요: ')

    run_calculations(G1, G2, current_string_number, current_fret_number, current_degree, increase_degree)


drop_string_number: 0, 프렛 번호: 0, 줄 번호: 6, 도수: 6, 음이름: E, 음정종류: 장
drop_string_number: 1, 프렛 번호: 2, 줄 번호: 5, 도수: 6, 음이름: B, 음정종류: 장
drop_string_number: 2, 프렛 번호: 4, 줄 번호: 4, 도수: 6, 음이름: F#, 음정종류: 장
drop_string_number: 3, 프렛 번호: 5, 줄 번호: 3, 도수: 6, 음이름: C, 음정종류: 장
drop_string_number: 4, 프렛 번호: 8, 줄 번호: 2, 도수: 6, 음이름: G, 음정종류: 장
drop_string_number: 5, 프렛 번호: 10, 줄 번호: 1, 도수: 6, 음이름: D, 음정종류: 장

현재 줄 번호: 6
현재 프렛 번호: 5
현재 도수: 4
올릴 도수: 3
